## 0. Dependencies

If you have not installed openCV, run the following code

In [ ]:
# ! pip install opencv-python

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import time
from sklearn.linear_model import LogisticRegression 
import matplotlib
from sklearn import metrics
from sklearn.metrics import precision_recall_curve

## 1. Subsetting and Loading

In [3]:
import cv2
import os
import random
import numpy as np

In [115]:
# set a seed
random.seed(328)

# define dir
data_dir = "/content/drive/My Drive/crop_part1"

# random sampling from 9780 samples
n = 9780
fnames_subset = random.sample(os.listdir(data_dir), n)

Construct y

In [116]:
y = [] # 0 (male) or 1 (female)
y_race = []
for fname in fnames_subset:
    temp = fname.split("_")
    y.append(int(temp[1]))
    if temp[2] in ("0", "1"):
        y_race.append(int(temp[2]))
    elif temp[2] in ("2", "3"):
        y_race.append(2)
    else:
        y_race.append(3)
    pass



In [ ]:
import pandas as pd
pd.Series(y).unique()

array([0, 1])

Construct X as np arrays

In [117]:
# X: flattened version
# X_origin_dict: original version; key: index, value: 3-D np array
X_rgb = list()
X_origin_dict = dict()
for i, fname in enumerate(fnames_subset):
    # construct dir
    dir = data_dir + "/" + fname
    
    # read the data
    dat = cv2.imread(dir)

    # convert to rgb
    dat = cv2.cvtColor(dat, cv2.COLOR_BGR2RGB)
    
    # store the original data
    X_origin_dict[i] = dat
    
    # store the data
    X_rgb.append(dat)




## 2. Preprocessing

In [92]:
for i, lab in enumerate(y):
    if lab == 3:
        delete = i

y.pop(delete)
y_race.pop(delete)
X_rgb.pop(delete)

NameError: ignored

In [118]:
# convert to np array
y = np.array(y)
y_race = np.array(y_race)
X_rgb = np.array(X_rgb)
print("The shape of the X_rgb is:", X_rgb.shape)

The shape of the X_rgb is: (3000, 200, 200, 3)


Subsetting for the expriments

In [93]:
import pandas as pd

In [119]:
df = pd.DataFrame({'y': y, 'y_race': y_race, 'img': list(X_rgb)}, columns=['y', 'y_race', 'img'])

In [51]:
df.head()

,y,y_race,img
0,0,3,"[[[150, 121, 107], [150, 121, 107], [149, 120,..."
1,0,3,"[[[210, 203, 197], [228, 221, 215], [245, 238,..."
2,0,3,"[[[31, 27, 16], [30, 26, 15], [29, 25, 14], [2..."
3,1,3,"[[[27, 17, 7], [24, 14, 4], [21, 12, 3], [19, ..."
4,0,3,"[[[16, 16, 16], [17, 17, 17], [18, 18, 18], [1..."


In [120]:
X_train_exp, X_test_exp, y_train_exp, y_test_exp = train_test_split(
    df, y, test_size=0.1, random_state=329
)

X_train_exp, X_val_exp, y_train_exp, y_val_exp = train_test_split(
    X_train_exp, y_train_exp, test_size=(0.1/0.9), random_state=525
)

In [54]:
X_train_exp.head(1)

,y,y_race,img
140,0,3,"[[[42, 42, 42], [34, 34, 34], [29, 29, 29], [3..."


In [68]:
X_train_exp["y_race"].unique()

array([0, 1, 3, 2])

Leave One Out

In [121]:
data_exp_r = {}
for _ in range(4):
    data_exp_r[_] = {}
    data_exp_r[_]["train"] = {}
    data_exp_r[_]["val"] = {}
    data_exp_r[_]["test"] = {}


for r in range(4):
    X_train0 = X_train_exp.loc[(X_train_exp["y_race"] != r)].copy()
    X_train_ex0 = X_train0["img"]
    y_train_ex0 = X_train0["y"]
    X_val0 = X_val_exp.loc[(X_val_exp["y_race"] != r)].copy()
    X_val_ex0 = X_val0["img"]
    y_val_ex0 = X_val0["y"]
    temp = []
    for _ in X_train_ex0:
        temp.append(np.array(_))
    data_exp_r[r]["train"]["X"] = np.array(temp)
    data_exp_r[r]["train"]["y"] = np.array(y_train_ex0)
    temp = []
    for _ in X_val_ex0:
        temp.append(np.array(_))    
    data_exp_r[r]["val"]["X"] = np.array(temp)
    data_exp_r[r]["val"]["y"] = np.array(y_val_ex0)

X_temp, y_temp = [], []
for i in range(4):
    X_test0 = X_test_exp.loc[(X_test_exp["y_race"] == i)].copy()
    X_test_r0 = X_test0["img"]
    y_test_r0 = X_test0["y"]
    temp = []
    for _ in X_test_r0:
        temp.append(np.array(_))  
    X_temp.append(np.array(temp))
    y_temp.append(np.array(y_test_r0))

for r in range(4):
    data_exp_r[r]["test"]["X"] = X_temp
    data_exp_r[r]["test"]["y"] = y_temp


In [109]:
type(data_exp_r[r]["train"]["X"])

numpy.ndarray

80% 20%

In [125]:
data_exp_g = {}

for _ in range(2):
    data_exp_g[_] = {}
    data_exp_g[_]["train"] = {}
    data_exp_g[_]["val"] = {}
    data_exp_g[_]["test"] = {}
  
for g in range(2):
    X_train0 = X_train_exp.loc[(X_train_exp["y"] == g)].copy()

    X_train_ex0 = X_train0["img"]
    y_train_ex0 = X_train0["y"]

    X_train1 = X_train_exp.loc[(X_train_exp["y"] != g)].copy()
    X_train_ex1 = X_train1["img"]
    y_train_ex1 = X_train1["y"]

    target = int(X_train_ex0.shape[0] * 0.25)

    X_slice = X_train_ex1[: target]
    y_slice = y_train_ex1[: target]
    
    X_exp_g = pd.concat([X_train_ex0, X_slice])
    y_exp_g = pd.concat([y_train_ex0, y_slice])
    temp = []
    for _ in X_exp_g:
        temp.append(np.array(_))
    temp = np.array(temp)
    data_exp_g[g]["train"]["X"] = temp
    data_exp_g[g]["train"]["y"] = y_exp_g

    # val
    X_val0 = X_val_exp.loc[(X_val_exp["y"] == g)].copy()
    X_val_ex0 = X_val0["img"]
    y_val_ex0 = X_val0["y"]

    X_val1 = X_val_exp.loc[(X_val_exp["y"] != g)].copy()
    X_val_ex1 = X_val1["img"]
    y_val_ex1 = X_val1["y"]

    target = int(X_val_ex0.shape[0] * 0.25)

    X_slice = X_val_ex1[: target]
    y_slice = y_val_ex1[: target]
    
    X_exp_g = pd.concat([X_val_ex0, X_slice])
    y_exp_g = pd.concat([y_val_ex0, y_slice])

    temp = []
    for _ in X_exp_g:
        temp.append(np.array(_))
    temp = np.array(temp)
    data_exp_g[g]["val"]["X"] = temp
    data_exp_g[g]["val"]["y"] = y_exp_g  

for g in range(2):
    X_temp = X_test_exp["img"]
    y_temp = X_test_exp["y"]
    temp = []
    for _ in X_temp:
        temp.append(np.array(_))
    temp = np.array(temp)
    data_exp_g[g]["test"]["X"] = temp
    data_exp_g[g]["test"]["y"] = np.array(y_temp)


In [126]:
data_exp_g[0]["test"]["X"].shape

(300, 200, 200, 3)